# AI - CA2 - Game - Mohamad Taha Fakharian

In [1]:
from random import random, shuffle
import copy
import time
import pandas as pd
from tqdm import tqdm

In [2]:
DEPTH = 3
PRUNE = False

In [3]:
class ConnectSin:
    YOU = 1
    CPU = -1
    EMPTY = 0
    DRAW = 0
    __CONNECT_NUMBER = 4
    board = None

    def __init__(self, board_size=(6, 7), silent=False):
        """
        The main class for the connect4 game

        Inputs
        ----------
        board_size : a tuple representing the board size in format: (rows, columns)
        silent     : whether the game prints outputs or not
        """
        assert len(board_size) == 2, "board size should be a 1*2 tuple"
        assert board_size[0] > 4 and board_size[1] > 4, "board size should be at least 5*5"

        self.columns = board_size[1]
        self.rows = board_size[0]
        self.silent = silent
        self.board_size = self.rows * self.columns
        
        # MiniMax stuff
        self.maximum_depth = DEPTH
        self.total_nodes_visited = 0
        
        self.last_row = 0
    def run(self, starter=None):
        """
        runs the game!

        Inputs
        ----------
        starter : either -1,1 or None. -1 if cpu starts the game, 1 if you start the game. None if you want the starter
            to be assigned randomly 

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that the game has drawn
        """
        if (not starter):
            starter = self.__get_random_starter()
        assert starter in [self.YOU, self.CPU], "starter value can only be 1,-1 or None"
        
        self.__init_board()
        turns_played = 0
        current_player = starter
        while(turns_played < self.board_size):
            
            if (current_player == self.YOU):
                self.__print_board()
                player_input = self.get_your_input()
            elif (current_player == self.CPU):
                player_input = self.__get_cpu_input()
            else:
                raise Exception("A problem has happend! contact no one, there is no fix!")
            if (not self.register_input(player_input, current_player)):
                self.__print("this move is invalid!")
                continue
            current_player = self.__change_turn(current_player)
            potential_winner = self.check_for_winners()
            turns_played += 1
            if (potential_winner != 0):
                self.__print_board()
                self.__print_winner_message(potential_winner)
                return potential_winner
        self.__print_board()
        self.__print("The game has ended in a draw!")
        return self.DRAW
    
    def __count_sequence(self, count, player):
        def vertical_seq(row, col):
            length = 0
            for i in range(row, self.rows):
                if self.board[i][col] == self.board[row][col]:
                    length += 1
                else:
                    break
            if length >= count:
                return 1
            else:
                return 0

        def horizontal_seq(row, col):
            length = 0
            for i in range(col, self.columns):
                if self.board[row][i] == self.board[row][col]:
                    length += 1
                else:
                    break
            if length >= count:
                return 1
            else:
                return 0

        def neg_diagonal_seq(row, col):
            length = 0
            j = col
            for i in range(row, -1, -1):
                if j >= self.columns:
                    break
                elif self.board[i][j] == self.board[row][col]:
                    length += 1
                else:
                    break
                j += 1 
            if length >= count:
                return 1
            else:
                return 0

        def pos_diagonal_seq(row, col):
            length = 0
            j = col
            for i in range(row, self.rows):
                if j >= self.columns:
                    break
                elif self.board[i][j] == self.board[row][col]:
                    length += 1
                else:
                    break
                j += 1 
            if length >= count:
                return 1
            else:
                return 0
        total = 0
        for i in range(self.rows):
            for j in range(self.columns):
                if self.board[i][j] == player:
                    total += vertical_seq(i, j)
                    total += horizontal_seq(i, j)
                    total += neg_diagonal_seq(i, j)
                    total += pos_diagonal_seq(i, j)
        return total
    
    def __value(self):
        winner = self.check_for_winners()
        if winner == self.YOU:
            return float('inf')
        elif winner == self.CPU:
            return float('-inf')
        
        your_score = 0
        cpu_score = 0
        base = 5
        current_base = base
        for count in range(2, self.__CONNECT_NUMBER):
            your_score += self.__count_sequence(count, self.YOU) * current_base
            cpu_score += self.__count_sequence(count, self.CPU) * current_base
            current_base = current_base * 10 + base
        return your_score - cpu_score
    
    def __minimax(self, move, current_depth, maximize, prune, alpha, beta):
        self.total_nodes_visited += 1
        moves = self.get_possible_moves()
        if len(moves) == 0 or current_depth == self.maximum_depth or self.check_for_winners() != 0:
            return self.__value(), move
        
        if prune:
            shuffle(moves)
        best_value = float('-inf') if maximize else float('inf')
        best_move = moves[0]
        current_player = self.YOU if maximize else self.CPU
        for i in moves:
            self.register_input(i, current_player)
            last_row = self.last_row
            candidate_value, candidate_move = self.__minimax(i, current_depth + 1, not maximize, prune, alpha, beta)
            self.board[last_row][i - 1] = 0
            
            if maximize and best_value < candidate_value:
                best_value = candidate_value
                best_move = i
                alpha = max(alpha, best_value)
                if beta <= alpha and prune:
                    break
 
            elif (not maximize) and best_value > candidate_value:
                best_value = candidate_value
                best_move = i
                beta = min(beta, best_value)
                if beta <= alpha and prune:
                    break
        return best_value, best_move
        
    def get_your_input(self):
        """
        gets your input

        Output
        ----------
        (int) an integer between 1 and column count. the column to put a piece in
        """
        alpha = float('-inf')
        beta = float('inf')
        value, move = self.__minimax(0, 0, True, PRUNE, alpha, beta)
        return move

    def check_for_winners(self):
        """
        checks if anyone has won in this position

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that nothing has happened
        """
        have_you_won = self.check_if_player_has_won(self.YOU)
        if have_you_won:
            return self.YOU
        has_cpu_won = self.check_if_player_has_won(self.CPU)
        if has_cpu_won:
            return self.CPU
        return self.EMPTY

    def check_if_player_has_won(self, player_id):
        """
        checks if player with player_id has won

        Inputs
        ----------
        player_id : the id for the player to check

        Output
        ----------
        (boolean) true if the player has won in this position
        """
        return (
            self.__has_player_won_diagonally(player_id)
            or self.__has_player_won_horizentally(player_id)
            or self.__has_player_won_vertically(player_id)
        )
    
    def is_move_valid(self, move):
        """
        checks if this move can be played

        Inputs
        ----------
        move : the column to place a piece in, in range [1, column count]

        Output
        ----------
        (boolean) true if the move can be played
        """
        if (move < 1 or move > self.columns):
            return False
        column_index = move - 1
        return self.board[0][column_index] == 0
    
    def get_possible_moves(self):
        """
        returns a list of possible moves for the next move

        Output
        ----------
        (list) a list of numbers of columns that a piece can be placed in
        """
        possible_moves = []
        for i in range(self.columns):
            move = i + 1
            if (self.is_move_valid(move)):
                possible_moves.append(move)
        return possible_moves
    
    def register_input(self, player_input, current_player):
        """
        registers move to board, remember that this function changes the board

        Inputs
        ----------
        player_input : the column to place a piece in, in range [1, column count]
        current_player: ID of the current player, either self.YOU or self.CPU

        """
        if (not self.is_move_valid(player_input)):
            return False
        self.__drop_piece_in_column(player_input, current_player)
        return True

    def __init_board(self):
        self.board = []
        for i in range(self.rows):
            self.board.append([self.EMPTY] * self.columns)

    def __print(self, message: str):
        if not self.silent:
            print(message)

    def __has_player_won_horizentally(self, player_id):
        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_vertically(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_diagonally(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __get_random_starter(self):
        players = [self.YOU, self.CPU]
        return players[int(random() * len(players))]
    
    def __get_cpu_input(self):
        """
        This is where clean code goes to die.
        """
        bb = copy.deepcopy(self.board)
        pm = self.get_possible_moves()
        for m in pm:
            self.register_input(m, self.CPU)
            if (self.check_if_player_has_won(self.CPU)):
                self.board = bb
                return m
            self.board = copy.deepcopy(bb)
        if (self.is_move_valid((self.columns // 2) + 1)):
            c = 0
            cl = (self.columns // 2) + 1
            for x in range(self.rows):
                if (self.board[x][cl] == self.CPU):
                    c += 1
            if (random() < 0.65):
                return cl
        return pm[int(random() * len(pm))]
    
    def __drop_piece_in_column(self, move, current_player):
        last_empty_space = 0
        column_index = move - 1
        for i in range(self.rows):
            if (self.board[i][column_index] == 0):
                last_empty_space = i
        self.board[last_empty_space][column_index] = current_player
        self.last_row = last_empty_space
        return True
        
    def __print_winner_message(self, winner):
        if (winner == self.YOU):
            self.__print("congrats! you have won!")
        else:
            self.__print("gg. CPU has won!")
    
    def __change_turn(self, turn):
        if (turn == self.YOU): 
            return self.CPU
        else:
            return self.YOU

    def __print_board(self):
        if (self.silent): return
        print("Y: you, C: CPU")
        for i in range(self.rows):
            for j in range(self.columns):
                house_char = "O"
                if (self.board[i][j] == self.YOU):
                    house_char = "Y"
                elif (self.board[i][j] == self.CPU):
                    house_char = "C"
                    
                print(f"{house_char}", end=" ")
            print()
            


In [ ]:
board_sizes_to_check = [(6,7), 
                        (7,8), 
                        (7,10)]

without_pruning = pd.DataFrame(columns = ['Board Size', 'Max Depth', 'Total Nodes visited', 'Win Chance', 'Time spent(s)'])
depths = [1, 3, 5]
runs = 200
i = 0
global PRUNE
PRUNE = False
for size in board_sizes_to_check:
    for depth in depths:
        global DEPTH
        DEPTH = depth
        t = 0
        nodes = 0
        wins = 0
        for run in tqdm(range(runs)):
            game = ConnectSin(board_size=size, silent=True)
            t0 = time.time()
            winner = game.run()
            t += time.time() - t0
            nodes += game.total_nodes_visited
            if winner == game.YOU:
                wins += 1
        without_pruning.loc[i] = [size, depth, nodes / runs, wins / runs, t / runs]
        i = i + 1

In [6]:
with_pruning = pd.read_csv('with.csv')

In [7]:
board_sizes_to_check = [(7,10)]

depths = [7]
runs = 200
i = 11
global PRUNE
PRUNE = True
for size in board_sizes_to_check:
    for depth in depths:
        global DEPTH
        DEPTH = depth
        t = 0
        nodes = 0
        wins = 0
        for run in tqdm(range(runs)):
            game = ConnectSin(board_size=size, silent=True)
            t0 = time.time()
            winner = game.run()
            t += time.time() - t0
            nodes += game.total_nodes_visited
            if winner == game.YOU:
                wins += 1
        with_pruning.loc[i] = [size, depth, nodes / runs, wins / runs, t / runs]
        i = i + 1

100%|█████████████████████████████████████████████████████████| 200/200 [5:04:46<00:00, 91.43s/it]


In [10]:
with_pruning.to_csv("with.csv", index=False)

In [26]:
mmd = pd.read_csv('with.csv')
mmd = mmd.append(with_pruning.loc[7])
mmd = mmd.append(with_pruning.loc[:2])
mmd

,Board Size,Max Depth,Total Nodes visited,Win Chance,Time spent(s)
0,"(6, 7)",1,32.795,0.505,0.003901
1,"(6, 7)",3,674.220,0.990,0.055208
2,"(6, 7)",5,10233.385,0.990,0.828597
3,"(6, 7)",7,133496.410,1.000,11.148775
4,"(7, 8)",1,35.535,0.610,0.005729
5,"(7, 8)",3,809.445,0.995,0.083688
6,"(7, 8)",5,15868.640,1.000,1.645075
7,"(7, 8)",7,259054.260,1.000,28.211300
0,"(7, 10)",1,40.730,0.755,0.007657
1,"(7, 10)",3,1155.610,0.995,0.143191


In [27]:
mmd.to_csv('with.csv', index=False)

In [11]:
board_sizes_to_check = [ (7,10)]

depths = [1, 3, 5]
runs = 200
i = 0
global PRUNE
PRUNE = True
for size in board_sizes_to_check:
    for depth in depths:
        global DEPTH
        DEPTH = depth
        t = 0
        nodes = 0
        wins = 0
        for run in tqdm(range(runs)):
            game = ConnectSin(board_size=size, silent=True)
            t0 = time.time()
            winner = game.run()
            t += time.time() - t0
            nodes += game.total_nodes_visited
            if winner == game.YOU:
                wins += 1
        with_pruning.loc[i] = [size, depth, nodes / runs, wins / runs, t / runs]
        i = i + 1

100%|███████████████████████████████████████████████████████████| 200/200 [13:45<00:00,  4.13s/it]


In [ ]:
board_sizes_to_check = [(6,7), 
                        (7,8), 
                        (7,10)]

with_pruning = pd.DataFrame(columns = ['Board Size', 'Max Depth', 'Total Nodes visited', 'Win Chance', 'Time spent(s)'])
depths = [1, 3, 5, 7]
runs = 200
i = 0
global PRUNE
PRUNE = True
for size in board_sizes_to_check:
    for depth in depths:
        global DEPTH
        DEPTH = depth
        t = 0
        nodes = 0
        wins = 0
        for run in tqdm(range(runs)):
            game = ConnectSin(board_size=size, silent=True)
            t0 = time.time()
            winner = game.run()
            t += time.time() - t0
            nodes += game.total_nodes_visited
            if winner == game.YOU:
                wins += 1
        with_pruning.loc[i] = [size, depth, nodes / runs, wins / runs, t / runs]
        i = i + 1